<a href="https://colab.research.google.com/github/JotaBlanco/CryptoAlgoTrading/blob/main/02%20-%20Features%20And%20Targets/02_Ejemplo_Variables_Volumen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 01 Petición a Messari

In [26]:
import requests
import pandas as pd
import time

def peticion_volumen_messari():
  api_key = "5fb492d2-32c3-4aac-99a6-6ba03e8f2fce"
  headers = { "x-messari-api-key": api_key }
  url = "https://data.messari.io/api/v1/assets"

  response = requests.get(url, headers)
  data = response.json()

  
  timestamp = data['status']['timestamp']
  df = pd.DataFrame()
  df['Timestamp'] = [pd.to_datetime(timestamp)]
  for i in range(len(data['data'])):
    dic_i = data['data'][i]
    df[dic_i['symbol']] = [dic_i['metrics']['blockchain_stats_24_hours']['transaction_volume']] # He cogido este volumen como ejemplo pero habría que ver cuál es el adecuado

  return df

In [27]:
df = peticion_volumen_messari()
df

,Timestamp,BTC,ETH,USDT,BNB,USDC,XRP,LUNA,ADA,SOL,...,DOT,BUSD,DOGE,UST,SHIB,MATIC,WBTC,CRO,DAI,ATOM
0,2022-03-17 07:49:41.186636490+00:00,2.281726e+10,1.003798e+10,324209.448944,None,6.495748e+09,9.103158e+08,None,2.675037e+10,None,...,8.898802e+07,4.539276e+08,1.843718e+08,None,None,None,6.155667e+08,1.133504e+07,7.471132e+08,None


In [28]:
# Fijaos lo que devuelve este código (la hora de hace 10 min). En base a esta línea se puede construír la función a continuación
pd.Timestamp.now() - pd.Timedelta(10, 'min')

Timestamp('2022-03-17 07:39:41.535910')

In [29]:
# Como lo que nos interesa es la diferencia del Volumen a medida que se va actualizando, vamos a hacer una función que devuelva la diferencia de volumen de los últimos 10 minutos

df_10mins = pd.DataFrame()

def diferencia_volumen_10_min(df, df_10mins):
  # Actualizamos df con la última fila recibida
  df_10mins = df_10mins.append(df)
  # Nos quedamos sólo con las filas de los últimos 10 mins
  df_10mins = df_10mins[df_10mins['Timestamp']>(pd.Timestamp.now(tz='UTC') - pd.Timedelta(10, 'min'))].reset_index(drop=True)
  # Hacemos un nuevo dataframe resultado de la última fila (último dato de volumen) - la primera fila (dato de hace 10 mins) 
  df_delta_vol = df_10mins.iloc[[0], 1:] - df_10mins.iloc[[0], 1:]
  return df_10mins, df_delta_vol

In [ ]:
df_10mins = pd.DataFrame()

while True:
  time.sleep(30)
  df = peticion_volumen_messari()
  print(df)
  df_10mins, df_delta_vol = diferencia_volumen_10_min(df, df_10mins)
  print(df_delta_vol)

                            Timestamp           BTC           ETH  \
0 2022-03-17 07:52:20.594213476+00:00  2.281726e+10  1.003798e+10   

            USDT   BNB          USDC           XRP  LUNA           ADA   SOL  \
0  324209.448944  None  6.495748e+09  9.103158e+08  None  2.675037e+10  None   

   ...           DOT          BUSD          DOGE   UST  SHIB MATIC  \
0  ...  8.898802e+07  4.539276e+08  1.843718e+08  None  None  None   

           WBTC           CRO           DAI  ATOM  
0  6.155667e+08  1.133504e+07  7.471132e+08  None  

[1 rows x 21 columns]
   BTC  ETH  USDT  BNB  USDC  XRP LUNA  ADA  SOL AVAX  DOT  BUSD  DOGE  UST  \
0  0.0  0.0   0.0  NaN   0.0  0.0  NaN  0.0  NaN  NaN  0.0   0.0   0.0  NaN   

  SHIB MATIC  WBTC  CRO  DAI ATOM  
0  NaN   NaN   0.0  0.0  0.0  NaN  
                            Timestamp           BTC           ETH  \
0 2022-03-17 07:52:51.516762350+00:00  2.281726e+10  1.003798e+10   

            USDT   BNB          USDC           XRP  LUNA      

# 02 Ejemplo código diccionario

In [1]:
import requests
import pandas as pd
import pytz
import datetime
import time

def get_crypto_volumen():
  url='https://api.coingecko.com/api/v3/simple/price?ids=bitcoin%2Ccardano%2Ctether%2Cethereum%2Csolana%2Cbinancecoin&vs_currencies=usd&include_24hr_vol=true&include_last_updated_at=true'
  response2=requests.get(url)
  print(response2)
  data=response2.json()
  df=pd.DataFrame()
  tz_Madrid = pytz.timezone('Europe/Madrid')
  datetime_Madrid = datetime.datetime.now(tz_Madrid)
  df['Timestamp']=[datetime_Madrid.strftime("%Y/%m/%d-%H:%M:%S")]
  for coin in range(len(data)):
    for moneda in data.keys():
      df[moneda]=int(data[moneda]['usd_24h_vol'])
  #cambiamos el nombre de las columnas para etiquetarlas y reconocerlas como el volumen en las ultimas 24h    
  new_columns = ['Timestamp', 'Vol_24h_ethereum', 'Vol_24h_bitcoin', 'Vol_24h_tether', 'Vol_24h_solana', 'Vol_24h_binancecoin', 'Vol_24h_cardano']
  df.columns = new_columns
  ##
  return df


In [22]:
df = get_crypto_volumen()
df

<Response [200]>


,Timestamp,Vol_24h_ethereum,Vol_24h_bitcoin,Vol_24h_tether,Vol_24h_solana,Vol_24h_binancecoin,Vol_24h_cardano
0,2022/03/16-20:31:38,1724203545,56697104217,1467048595,34535486731,19314019117,944500902


In [27]:
dic_volumen = {}


while True:
  df = get_crypto_volumen()
  timestamp_i = df['Timestamp'].iloc[0]
  for col_i in df.columns[1:]:
    if col_i not in dic_volumen.keys():
      dic_volumen[col_i] = pd.DataFrame({'Timestamp':[timestamp_i], 
                                        'Volumen24h': [df[col_i].iloc[0]]})
      
    elif df[col_i].iloc[0] != dic_volumen[col_i]['Volumen24h'].iloc[-1]:
      print(col_i + " actualizado")
      dic_volumen[col_i] = dic_volumen[col_i].append(
          pd.DataFrame({'Timestamp':[timestamp_i], 'Volumen24h': [df[col_i].iloc[0]]}))
    
    time.sleep(2)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
Vol_24h_ethereum actualizado
Vol_24h_bitcoin actualizado
Vol_24h_tether actualizado
Vol_24h_solana actualizado
Vol_24h_binancecoin actualizado
Vol_24h_cardano actualizado
<Response [200]>
<Response [200]>
Vol_24h_ethereum actualizado
Vol_24h_bitcoin actualizado
Vol_24h_tether actualizado
Vol_24h_solana actualizado
Vol_24h_binancecoin actualizado
Vol_24h_cardano actualizado
<Response [200]>
Vol_24h_ethereum actualizado
Vol_24h_bitcoin actualizado
Vol_24h_tether actualizado
Vol_24h_solana actualizado
Vol_24h_binancecoin actualizado
Vol_24h_cardano actualizado
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


KeyboardInterrupt: ignored

In [28]:
for vol_i in dic_volumen.keys():
  print(vol_i)
  display(dic_volumen[vol_i])

Vol_24h_ethereum


,Timestamp,Volumen24h
0,2022/03/16-20:39:10,57141760273
0,2022/03/16-20:39:58,1752127681
0,2022/03/16-20:40:22,57141760273
0,2022/03/16-20:40:34,1752127681


Vol_24h_bitcoin


,Timestamp,Volumen24h
0,2022/03/16-20:39:10,1751885404
0,2022/03/16-20:39:58,34967580537
0,2022/03/16-20:40:22,1751885404
0,2022/03/16-20:40:34,34967580537


Vol_24h_tether


...

,Timestamp,Volumen24h
0,2022/03/16-20:39:10,946394704
0,2022/03/16-20:39:58,946620022
0,2022/03/16-20:40:22,946394704
0,2022/03/16-20:40:34,946620022


Vol_24h_binancecoin


,Timestamp,Volumen24h
0,2022/03/16-20:39:10,34908568816
0,2022/03/16-20:39:58,19469257819
0,2022/03/16-20:40:22,34908568816
0,2022/03/16-20:40:34,19469257819


Vol_24h_cardano


,Timestamp,Volumen24h
0,2022/03/16-20:39:10,1513976006
0,2022/03/16-20:39:58,1499652668
0,2022/03/16-20:40:22,1513976006
0,2022/03/16-20:40:34,1499652668
